In [20]:
y

Model has 99k params
Model has 99k params


ZeroDivisionError: division by zero

In [25]:
def generate_text(input: str):
    max_tokens = 1000
    input_tokens = tokenise(input)
    print(input, end='')
    
    for i in range(max_tokens):
        now = model(input_tokens.unsqueeze(0))[0][-1]
        now = F.softmax(now, dim= 0)
        token = torch.multinomial(now,1).item()
        input_tokens = torch.tensor(input_tokens.tolist() + [token])
        text = decode([token])
        print(text, end='')
        input_tokens = input_tokens[-context_length:]
                

generate_text("you")  

you thick.

Gul
They by nothy marows and yourew'tre mus'der of grow of kiver of to havom a a strutus wis hive!

Sut gime wan in my the do-was, notr borsher alle furt,
Lounier, to of you. Haudsed, dida anon Beasse they sely were the eysird
I raiur like
trine, Aurcke bear is bealos is varcemp,
To sones our rie, not all tehour if a hou?
If antireld, in arf. Toud you feal ir a rudvereate in hart to life
And sove all bust you wil.

Trespen of im, you! is nother uin the in the bein oble,
Whou of I MI'll, a is have dedl-dighs procet'stipigh for my oftucte.

Haucterlive lenotion cor.

MAERD INCI:
'Tt, of, I far the wherrieds, all may thy wrar.

KING O
RINCIO:
Alad puccar they thou grung bear pan my I and mere chis bard to fill hich his be chom you love thingly the in thou eake:
And antr! Hadere Pre it.

MERY RICEO:
Yood didan:
I kis himen kivensse what wackes! our may in ale!

NREO:
My st and-deatd, and houbbles wild confiringle vive; die;
kniry: lich cawgh I the wich bedragion a be noth we Ca

In [11]:
generate_text(" ")  

 whis bens,
Whis thirgar tink for and by eee acaing ifs fall andoy thow And wommith to theame?

Rome hatheut?

BENVO:
Mill tak. And Romparn the the dell sor thisild,
Heake lus nowell theur, who say anto arier this Waite,
And arte the orrest their flor eneque?

PRICICK:
I you
Tord, brees Marred thou ancel.

'lance; cas and it misess worsly fis.

EDWABES:
Nure muel
Tooder:
I her. Ray kide mall bour gis, ase, and is is us cady--y the hese we younor he it slooks hade to meicke:
'botion nut
ETarids whath. That she stre homse him,
Yo migh creast! liftotord your is hive:
I will agarin I as beake; and in you ar poner,
But in that and thaul aurt bretle fore I propursed
CYodes noth are my as him feal,
No wregns, and your thall gend eeford.

PAPURIANUS:
Whe wo worth
So kim you faul the thee but the bing
Kirne bee!-Hay forcher's as naye his
Whicl say both'd reid bive; and the thickel geet
Aw o
So mor's the proubstervicarting Buse.

Ply axtid ye twill firfaires irvettlal yere;
Away, where itrances!

In [22]:
a = tokenise("alik").unsqueeze(0)
output = model(a)
F.cross_entropy(output.resize(4,65), F.one_hot(tokenise("like").unsqueeze(0), vocab_size).resize(4, 65).float())

tensor(2.3337, grad_fn=<DivBackward1>)

In [15]:
PATH = "models/pavan_gpt_100k_1.91.bin"
torch.save(model.state_dict(), PATH)
